In [1]:
! pip install pymupdf    pdf2image


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 32.3 MB/s eta 0:00:00


In [11]:

import os
import pathlib
import textwrap
from PIL import Image


import google.generativeai as genai


genai.configure(api_key="AIzaSyB9YV9z1hKhiYlqnUerhpuXMG")


def get_gemini_response(input,image,prompt):
    model = genai.GenerativeModel('gemini-2.0-flash-001')
    response = model.generate_content([input,image[0],prompt])
    return response.text

In [4]:

input_prompt = """
               You are an expert in understanding scan reports.
               You will receive input images as scan and medical reports &
               you will have to answer questions based on the input image
               """

In [12]:
uploaded_file = "/content/NORMAL-NT-REPORT-1st-Part-1-889x1024.webp"
with open(uploaded_file, "rb") as f:
    # Read the entire content of the file into bytes
    # instead of using getvalue()
    bytes_data = f.read()

    image_parts = [
        {
            "mime_type": "image/webp",  # Specify the mime type directly
            "data": bytes_data
        }
    ]
    image_data = image_parts
# Use a different variable name for the prompt, like 'user_prompt'
user_prompt = "Tell me about the scan"  # Replace with the actual prompt
response = get_gemini_response(input_prompt, image_data, user_prompt)

In [13]:
import re
clean_text = re.sub(r'\n+', '\n', response).strip()

print(clean_text)

"Here's a summary of the scan report:\n\n**Patient Information:**\n\n*   Age/Sex: 31 Years / Female\n*   LMP Date: 02/03/2023\n*   LMP EDD: 07/12/2023 (12 Weeks 1 Day)\n*   Visit Number: 1\n\n**Indications:**\n\n*   First trimester screening\n\n**Procedure:**\n\n*   Real-time B-mode ultrasonography of the gravid uterus was performed.\n*   Route: Transabdominal and Transvaginal\n\n**Findings:**\n\n*   Single intrauterine gestation.\n\n**Medical Notes:**\n\n*   Blood group: A1B+ve\n*   Height: 159 cm\n*   Weight: 48.2 kg\n*   Marital History: 4 years, Non-consanguineous marriage\n*   Menstrual History: Regular\n*   Gravida: 2, Para: 1, Live: 1, Abortion: 0\n*   Significant previous obstetric details: Nil\n*   Medical/Surgical History: Lscs\n\n**Maternal:**\n\n*   Cervix measured 3.10 cm in length.\n*   Right Uterine: 1.8\n*   Left Uterine: 1.4\n*   Mean PI: 1.6\n\n**Fetus:**\n\n*   Placenta: Anterior\n*   Liquor: Normal\n*   Fetal activity: Present\n*   Cardiac activity: Present\n*   Fet

In [16]:
!pip install --upgrade google-generativeai

In [3]:
import google.generativeai as genai
import fitz  # PyMuPDF for text extraction
from pdf2image import convert_from_path  # Convert PDF pages to images
from PIL import Image
import io

# 🔹 Configure Gemini API (Replace with your API Key)
genai.configure(api_key="AIzaSyB9YV9z1hKhiYlqnUeMG1dZahGEYI")

def extract_pdf_text(pdf_path):
    """Extract text from a PDF."""
    doc = fitz.open(pdf_path)  # Open PDF
    text_data = [page.get_text("text") for page in doc]  # Extract text
    return "\n".join(text_data)  # Combine all text

def extract_pdf_images(pdf_path):
    """Extract images from a PDF."""
    doc = fitz.open(pdf_path)  # Open PDF
    images = []

    for page in doc:
        for img in page.get_images(full=True):
            xref = img[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            image = Image.open(io.BytesIO(img_bytes))
            images.append(image)

    return images  # List of images

def ask_gemini_llm(pdf_text, question):
    """Ask a question to Gemini Pro (Text-only LLM)."""
    model = genai.GenerativeModel("gemini-1.5-pro")  # Text LLM
    response = model.generate_content([pdf_text, question])
    return response.text  # Return LLM response

def analyze_images_with_vlm(images):
    """Analyze images using Gemini Vision (VLM)."""
    if not images:
        return "No images found in the PDF."

    model = genai.GenerativeModel("gemini-2.0-flash-001")

    image_parts = []
    for img in images:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=img.format)
        img_byte_arr = img_byte_arr.getvalue()

        image_parts.append({
            "mime_type": "image/" + img.format.lower(),
            "data": img_byte_arr
        })


    response = model.generate_content(["Analyze these images and summarize their content:"] + image_parts)

    return response.text


pdf_path = "/content/DeepSeek_R1.pdf"

pdf_text = extract_pdf_text(pdf_path)
pdf_images = extract_pdf_images(pdf_path)

image_analysis = analyze_images_with_vlm(pdf_images)




Ask a question about the PDF (or type 'exit' to quit): tell me about the benchmark

🔹 Text-Based Answer: The benchmark in the DeepSeek-R1 paper is a comprehensive suite of tests designed to evaluate the reasoning capabilities and general performance of Large Language Models (LLMs).  It covers a wide range of tasks, including:

**Reasoning-focused:**

* **Math:** AIME 2024, MATH-500, CNMO 2024 (Chinese National High School Mathematics Olympiad). These test the model's ability to solve complex mathematical problems, often requiring multi-step reasoning and logical deduction.
* **Coding:** Codeforces, LiveCodeBench, SWE-Bench Verified, Aider.  These assess coding proficiency in various aspects, from competitive programming (Codeforces) to practical software engineering tasks (SWE-Bench, Aider) and code generation (LiveCodeBench).

**Knowledge and General Capabilities:**

* **General Knowledge:** MMLU, MMLU-Redux, MMLU-Pro, GPQA Diamond, SimpleQA, C-SimpleQA. These evaluate the model's un

In [7]:
while True:
    user_question = input("\nAsk a question about the PDF (or type 'exit' to quit): ")
    if user_question.lower() == "exit":
        break

    text_answer = ask_gemini_llm(pdf_text, user_question)

    print("\n🔹 Text-Based Answer:", text_answer)
    print("\n🔹 Image Analysis (VLM):", image_analysis)


Ask a question about the PDF (or type 'exit' to quit): tell me about the images

🔹 Text-Based Answer: The image you provided appears to be a screenshot of a research paper titled "DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning" by DeepSeek-AI.  The screenshot includes the abstract, table of contents, introduction, and a figure showing benchmark performance.

Key takeaways from the visible sections:

* **Focus on Reasoning:** The paper introduces DeepSeek-R1, a language model designed to improve reasoning capabilities.  They've developed two versions: DeepSeek-R1-Zero, trained purely through reinforcement learning (RL), and DeepSeek-R1, which incorporates cold-start data and a multi-stage training process.
* **Reinforcement Learning Emphasis:**  A major contribution is the use of large-scale RL to improve reasoning, even without initial supervised fine-tuning.  DeepSeek-R1-Zero demonstrates this capability, though it suffers from readability and lang

In [5]:
pdf_text = extract_pdf_text(pdf_path)

# 🔹 Print the extracted text
print("\n🔹 Extracted Text from PDF:\n")
print(pdf_text[:1000])  # Print the first 1000 characters (to avoid too much output)



🔹 Extracted Text from PDF:

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) disti

In [6]:
pdf_images = extract_pdf_images(pdf_path)

# 🔹 Show extracted images
print(f"\n🔹 Extracted {len(pdf_images)} images from the PDF.")

# Display the first few images
for i, img in enumerate(pdf_images[:5]):  # Show only first 5 images
    img.show(title=f"Extracted Image {i+1}")  # Opens image in the default viewer



🔹 Extracted 2 images from the PDF.


In [8]:
for i, img in enumerate(pdf_images):
    img.save(f"extracted_image_{i+1}.png")  # Save as PNG
    print(f"Saved extracted_image_{i+1}.png")


Saved extracted_image_1.png
Saved extracted_image_2.png


In [ ]:
from google.colab import drive
drive.mount('/content/drive')